In [4]:
with open("/home/serdar/Documents/medp/logs/ozet_20250315_201717.txt") as f:
    lines = f.read()
import re
# <üniversite></üniversite> arasındaki herşeyi alır
üniversiteler = re.findall(r'<üniversite>(.*?)</üniversite>', lines, re.DOTALL)
len(üniversiteler)

156

In [5]:
import sys
import time
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_driver(url, max_retries=3):
    """
    Undetected Chrome driver oluşturur ve belirtilen URL'ye gider.
    Hatalar durumunda tekrar deneme yapar.
    """
    for attempt in range(max_retries):
        try:
            ua = UserAgent()
            options = uc.ChromeOptions()
            options.add_argument(f'user-agent={ua.random}')
            driver = uc.Chrome(options=options)
            driver.get(url)
            # Sayfanın yüklenmesi için bekle
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            return driver
        except (WebDriverException, TimeoutException) as e:
            if attempt < max_retries - 1:
                print(f"Driver oluşturma hatası: {e}. Tekrar deneniyor...")
                time.sleep(2)
            else:
                print(f"Driver oluşturulamadı: {e}")
                raise
    return None

def click_tarama(driver):
    """Tarama sekmesine tıklar"""
    try:
        tarama_path = '//*[@id="navigation2"]/ul/li[2]/a'
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, tarama_path))
        )
        element.click()
        return True
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Tarama sekmesi tıklama hatası: {e}")
        return False
    
def üniversite_seç(driver, üniversite):
    """Belirtilen üniversiteyi seçer"""
    try:
        path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[2]/input[1]'
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, path))
        ).click()
        
        # Yeni pencereye geç
        driver.switch_to.window(driver.window_handles[1])
        
        # Üniversite link metnine tıkla
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, üniversite))
        ).click()
        
        # Ana pencereye geri dön
        driver.switch_to.window(driver.window_handles[0])
        return True
    except (TimeoutException, NoSuchElementException, IndexError) as e:
        print(f"Üniversite seçme hatası: {e}")
        return False

def click_tez_türü(driver, tür="Tıpta Uzmanlık"):
    """Tez türünü seçer"""
    try:
        tez_türü_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[4]/select'
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, tez_türü_path))
        )
        element.click()
        element.send_keys(tür)
        return True
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Tez türü seçme hatası: {e}")
        return False

def click_bul(driver):
    """Bul butonuna tıklar"""
    try:
        bul_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[8]/td/input[3]'
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, bul_path))
        ).click()
        return True
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Bul butonuna tıklama hatası: {e}")
        return False
    
def sonuç_sayısı(driver):
    """Sonuç sayısını bulur ve döndürür"""
    try:
        # Sonuç mesajının yüklenmesini bekle
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "divuyari"))
        )
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        sonuç_mesajı = soup.find("div", {"id": "divuyari"}).text
        sayılar = re.findall(r'\d+', sonuç_mesajı)
        
        if not sayılar:
            return 0
        return int(sayılar[0])
    except (TimeoutException, AttributeError, IndexError) as e:
        print(f"Sonuç sayısı bulma hatası: {e}")
        return 0

def yıl_aralığı(driver, yıl1, yıl2):
    """Yıl aralığını belirler"""
    try:
        # yıl1'in yıl2'den küçük olmasını sağla
        if yıl1 > yıl2:
            yıl1, yıl2 = yıl2, yıl1
            
        yıl1_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[6]/select[1]'
        yıl2_path = '//*[@id="tabs-1"]/form/table/tbody/tr/td/table/tbody/tr[2]/td[6]/select[2]'
        
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, yıl1_path))
        ).send_keys(str(yıl1))
        
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, yıl2_path))
        ).send_keys(str(yıl2))
        
        return True
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Yıl aralığı belirleme hatası: {e}")
        return False
    
def click_process(driver, üni, yıl1=0, yıl2=0):
    """Arama sürecini yönetir"""
    try:
        if not click_tarama(driver):
            return 0
        
        if not üniversite_seç(driver, üni):
            return 0
            
        if not click_tez_türü(driver):
            return 0
            
        if yıl1 != 0 and yıl2 != 0:
            if not yıl_aralığı(driver, yıl1, yıl2):
                return 0
                
        if not click_bul(driver):
            return 0
            
        # Sonuçların yüklenmesini bekle
        time.sleep(2)
        
        return sonuç_sayısı(driver)
    except Exception as e:
        print(f"Arama işlemi hatası: {e}")
        return 0

def get_table(driver):
    """Tez tablosunu alır ve DataFrame'e çevirir"""
    try:
        # Sonuçların tamamen yüklenmesini bekle
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "script"))
        )
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # JavaScript veri yapısını bul - sayfadaki 6. script etiketini hedefle
        script_tags = soup.find_all("script", {"type": "text/javascript"})
        if len(script_tags) < 6:
            print(f"Script etiketleri bulunamadı (bulunan: {len(script_tags)})")
            return pd.DataFrame()
            
        data = str(script_tags[5])
        
        # Regex ile veri alanlarını çıkar
        tez_no = re.findall(r'(?<=\)>)\d+(?=</span>)', data)
        yazar = re.findall(r'(?<=name: ")(.*?)(?=",)', data)
        yıl = re.findall(r'(?<=age: ")\d+(?=",)', data)
        title = re.findall(r'(?<=weight: ")(.*?)(?=</span>)', data)
        üniversite = re.findall(r'(?<=uni:")(.*?)(?=",)', data)
        tez_türü = re.findall(r'(?<=important: ")(.*?)(?=",)', data)
        konu = re.findall(r'(?<=someDate: ")(.*?)(?=",)', data)
        
        # Sonuçların tutarlı olmasını sağla
        min_length = min(len(tez_no), len(yazar), len(yıl), len(title), 
                         len(üniversite), len(tez_türü), len(konu))
        
        if min_length == 0:
            print("Tablo verisi bulunamadı")
            return pd.DataFrame()
            
        # Tüm listeleri aynı uzunluğa getir
        tez_no = tez_no[:min_length]
        yazar = yazar[:min_length]
        yıl = yıl[:min_length]
        title = title[:min_length]
        üniversite = üniversite[:min_length]
        tez_türü = tez_türü[:min_length]
        konu = konu[:min_length]
        
        # DataFrame oluştur
        return pd.DataFrame({
            "tez_no": tez_no,
            "yazar": yazar,
            "yıl": yıl,
            "title": title,
            "üniversite": üniversite,
            "tez_türü": tez_türü,
            "konu": konu
        })
        
    except Exception as e:
        print(f"Tablo alma hatası: {e}")
        return pd.DataFrame()

In [6]:
# Main.py'nin en başına ekleyebilirsiniz
import os
if not os.path.exists("logs"):
    os.makedirs("logs")
    print("'logs' dizini manuel olarak oluşturuldu.")

In [7]:
import os
import sys
import time
import pandas as pd
import logging
from datetime import datetime
from functions import *

# Log dosyası oluşturma
def setup_logging():
    """Log sistemi oluşturur"""
    # Log dizini kontrolü
    if not os.path.exists("logs"):
        os.makedirs("logs")
        print("'logs' dizini oluşturuldu.")
    
    # Log dosyası adı (tarih-saat)
    log_filename = f"logs/tez_scraping_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    
    # Log formatı
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    
    logging.info(f"Log dosyası oluşturuldu: {log_filename}")
    return log_filename

def add_to_failed_universities(üni, error_message=None):
    """Başarısız üniversiteyi failed_universities.txt dosyasına ekler"""
    try:
        with open("failed_universities.txt", "a", encoding="utf-8") as f:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if error_message:
                f.write(f"{timestamp} - <üniversite>{üni}</üniversite> - Hata: {error_message}\n")
            else:
                f.write(f"{timestamp} - <üniversite>{üni}</üniversite>\n")
        logging.info(f"{üni} başarısız üniversiteler listesine eklendi.")
    except Exception as e:
        logging.error(f"Başarısız üniversiteler dosyasına yazma hatası: {e}")

def process_university(üni, başlangıç_yılı=2025):
    """
    Belirli bir üniversite için tez verilerini işler ve kaydeder.
    
    Args:
        üni (str): İşlenecek üniversite adı
        başlangıç_yılı (int): Başlangıç yılı (varsayılan: 2025)
    
    Returns:
        bool: İşlem başarılı ise True, değilse False
    """
    logging.info(f"{'='*50}")
    logging.info(f"{üni} üniversitesi işleniyor...")
    logging.info(f"{'='*50}")
    
    url = "https://tez.yok.gov.tr/UlusalTezMerkezi/"
    driver = None
    
    try:
        # Data dizini kontrolü
        if not os.path.exists("data"):
            os.makedirs("data")
            logging.info("'data' dizini oluşturuldu.")
        
        # Driver oluştur ve arama yap
        driver = get_driver(url)
        sonuç = click_process(driver, üni)
        
        if sonuç == 0:
            logging.info(f"{üni} için tıpta uzmanlık tezi bulunmadı. Bu bir hata değil.")
            if driver:
                driver.close()
            return True  # Sıfır sonuç normal bir durum, başarısız sayılmaz
            
        logging.info(f"{üni} için toplam sonuç sayısı: {sonuç}")
        
        # Sonuç sayısına göre işlem stratejisini belirle
        if sonuç < 2000:
            # 2000'den az sonuç - tek seferde işle
            process_results(driver, üni)
        else:
            # 2000'den fazla sonuç - yıl aralıklarına böl
            if driver:
                driver.close()
            
            # Döngü parametrelerini belirle
            if sonuç < 4000:
                yıl_aralığı_adımı = 12  # Her 12 yılda bir böl
                döngü_sayısı = sonuç // 2000 + 2
            else:
                yıl_aralığı_adımı = 6   # Her 6 yılda bir böl
                döngü_sayısı = sonuç // 1000 + 2
                
            process_by_year_ranges(üni, başlangıç_yılı, yıl_aralığı_adımı, döngü_sayısı, url)
            
        logging.info(f"{üni} için işlem başarıyla tamamlandı.")
        return True
        
    except Exception as e:
        error_msg = f"HATA: {üni} işlenirken bir hata oluştu: {e}"
        logging.error(error_msg)
        add_to_failed_universities(üni, str(e))
        if driver:
            try:
                driver.close()
            except:
                pass
        return False

def process_results(driver, üni, yıl1=None, yıl2=None):
    """
    Arama sonuçlarını işler ve CSV'ye kaydeder.
    
    Args:
        driver: Selenium WebDriver nesnesi
        üni (str): Üniversite adı
        yıl1 (int, optional): Başlangıç yılı
        yıl2 (int, optional): Bitiş yılı
    
    Returns:
        bool: İşlem başarılı ise True, değilse False
    """
    try:
        # Tabloyu al
        df = get_table(driver)
        
        if df.empty:
            logging.warning("Tablo verisi alınamadı.")
            add_to_failed_universities(üni, "Tablo verisi alınamadı")
            driver.close()
            return False
            
        logging.info(f"Toplam {len(df)} adet tez verisi alındı.")
        
        # Dosya adını belirle
        if yıl1 and yıl2:
            filename = f"data/{üni}_{yıl1}_{yıl2}.csv"
        else:
            filename = f"data/{üni}.csv"
            
        # CSV'ye kaydet
        df.to_csv(filename, index=False)
        logging.info(f"Veriler {filename} dosyasına kaydedildi.")
        
        # Driver'ı kapat
        driver.close()
        return True
        
    except Exception as e:
        error_msg = f"Sonuçları işlerken hata: {e}"
        logging.error(error_msg)
        add_to_failed_universities(üni, error_msg)
        try:
            driver.close()
        except:
            pass
        return False

def process_by_year_ranges(üni, başlangıç_yılı, yıl_adımı, döngü_sayısı, url):
    """
    Belirli yıl aralıklarında bölünmüş şekilde verileri işler.
    
    Args:
        üni (str): Üniversite adı
        başlangıç_yılı (int): Başlangıç yılı
        yıl_adımı (int): Her adımda azaltılacak yıl sayısı
        döngü_sayısı (int): Kaç yıl aralığı oluşturulacağı
        url (str): YÖK tez merkezi URL'si
    
    Returns:
        bool: İşlem başarılı ise True, değilse False
    """
    current_year = başlangıç_yılı
    success_count = 0
    
    for i in range(1, döngü_sayısı):
        # Yıl aralığını belirle
        if i == döngü_sayısı - 1:
            yıl1 = 1900  # Son aralık için 1900'den başla
        else:
            yıl1 = current_year - yıl_adımı
            
        yıl2 = current_year
        
        logging.info(f"\nYıl aralığı: {yıl1}-{yıl2} işleniyor...")
        
        driver = None
        try:
            # Bu yıl aralığı için driver oluştur ve arama yap
            driver = get_driver(url)
            sonuç = click_process(driver, üni, yıl1=yıl1, yıl2=yıl2)
            
            if sonuç == 0:
                logging.info(f"{üni} için {yıl1}-{yıl2} aralığında tez bulunmadı.")
                if driver:
                    driver.close()
                continue
                
            logging.info(f"{üni} için {yıl1}-{yıl2} aralığında {sonuç} sonuç bulundu.")
            
            # Sonuçları işle
            if process_results(driver, üni, yıl1, yıl2):
                success_count += 1
        
        except Exception as e:
            error_msg = f"{üni} için {yıl1}-{yıl2} aralığında hata oluştu: {e}"
            logging.error(error_msg)
            add_to_failed_universities(üni, f"Yıl aralığı {yıl1}-{yıl2}: {str(e)}")
            if driver:
                try:
                    driver.close()
                except:
                    pass
            
        # Bir sonraki aralık için yılları güncelle
        current_year = yıl1 - 1
    
    logging.info(f"{üni} için {döngü_sayısı-1} aralıktan {success_count} tanesi başarıyla işlendi.")
    return success_count > 0


def main():
    """Ana program akışı"""
    # Log sistemi oluştur
    log_file = setup_logging()
    
    # failed_universities.txt dosyasının başına başlık ekle
    try:
        with open("failed_universities.txt", "w", encoding="utf-8") as f:
            f.write(f"# Başarısız Üniversiteler - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("# Format: Tarih Saat - <üniversite>Üniversite Adı</üniversite> - Hata Mesajı\n")
            f.write("-" * 80 + "\n")
    except Exception as e:
        logging.error(f"Failed universities dosyası oluşturma hatası: {e}")
    
    try:
        # Üniversite listesini oku
        # üniversiteler = pd.read_csv("üniversiteler.csv")['üniversiteler'].tolist() # Üniversite listesi csvden alınacak
        
        logging.info(f"Toplam {len(üniversiteler)} üniversite işlenecek.")
        
        # Başarılı ve başarısız listeleri
        başarılı_üniler = []
        başarısız_üniler = []
        
        # Her üniversite için işlem yap
        for üni in üniversiteler[8:]:
            if process_university(üni, başlangıç_yılı=2025):
                başarılı_üniler.append(üni)
            else:
                başarısız_üniler.append(üni)
                
            # Üniversiteler arası kısa bekleme
            time.sleep(2)
        
        # Özet log
        logging.info(f"\n{'='*50}")
        logging.info(f"İŞLEM ÖZET RAPORU")
        logging.info(f"{'='*50}")
        logging.info(f"Toplam üniversite sayısı: {len(üniversiteler)}")
        logging.info(f"Başarılı işlenen üniversite sayısı: {len(başarılı_üniler)}")
        logging.info(f"Başarısız üniversite sayısı: {len(başarısız_üniler)}")
        
        if başarısız_üniler:
            logging.warning("Başarısız üniversiteler:")
            for üni in başarısız_üniler:
                logging.warning(f"<üniversite>{üni}</üniversite>")
            
        # Özet dosyası
        özet_file = f"logs/ozet_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        with open(özet_file, 'w', encoding='utf-8') as f:
            f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Log dosyası: {log_file}\n\n")
            f.write(f"Toplam üniversite sayısı: {len(üniversiteler)}\n")
            f.write(f"Başarılı işlenen üniversite sayısı: {len(başarılı_üniler)}\n")
            f.write(f"Başarısız üniversite sayısı: {len(başarısız_üniler)}\n\n")
            
            f.write("Başarılı üniversiteler:\n")
            for i, üni in enumerate(başarılı_üniler, 1):
                f.write(f"{i}. {üni}\n")
            
            f.write("\nBaşarısız üniversiteler:\n")
            for i, üni in enumerate(başarısız_üniler, 1):
                f.write(f"{i}. <üniversite>{üni}</üniversite>\n")
        
        logging.info(f"Özet rapor {özet_file} dosyasına kaydedildi.")
        logging.info(f"Başarısız üniversiteler 'failed_universities.txt' dosyasına kaydedildi.")
        logging.info("\nTüm üniversiteler için işlem tamamlandı!")
        
    except Exception as e:
        logging.error(f"Ana program hatası: {e}")
        return False


if __name__ == "__main__":
    main()

2025-03-16 03:00:29,769 - INFO - Log dosyası oluşturuldu: logs/tez_scraping_20250316_030029.log
2025-03-16 03:00:29,770 - INFO - Toplam 156 üniversite işlenecek.
2025-03-16 03:00:29,770 - INFO - ==================================================
2025-03-16 03:00:29,771 - INFO - ALANYA ÜNİVERSİTESİ üniversitesi işleniyor...
2025-03-16 03:00:29,772 - INFO - ==================================================
2025-03-16 03:00:32,419 - INFO - patching driver executable /home/serdar/.local/share/undetected_chromedriver/undetected_chromedriver
2025-03-16 03:00:49,741 - INFO - ALANYA ÜNİVERSİTESİ için tıpta uzmanlık tezi bulunmadı. Bu bir hata değil.
2025-03-16 03:00:51,809 - INFO - ==================================================
2025-03-16 03:00:51,810 - INFO - ANKARA BİLİM ÜNİVERSİTESİ üniversitesi işleniyor...
2025-03-16 03:00:51,810 - INFO - ==================================================
2025-03-16 03:00:54,340 - INFO - patching driver executable /home/serdar/.local/share/undetected

In [8]:
# import os
# import sys
# import time
# import pandas as pd
# import logging
# from datetime import datetime
# from functions import *

# # Log dosyası oluşturma
# def setup_logging():
#     """Log sistemi oluşturur"""
#     # Log dizini kontrolü
#     if not os.path.exists("logs"):
#         os.makedirs("logs")
#         print("'logs' dizini oluşturuldu.")
    
#     # Log dosyası adı (tarih-saat)
#     log_filename = f"logs/tez_scraping_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    
#     # Log formatı
#     logging.basicConfig(
#         level=logging.INFO,
#         format='%(asctime)s - %(levelname)s - %(message)s',
#         handlers=[
#             logging.FileHandler(log_filename),
#             logging.StreamHandler()
#         ]
#     )
    
#     logging.info(f"Log dosyası oluşturuldu: {log_filename}")
#     return log_filename

# def add_to_failed_universities(üni, error_message=None):
#     """Başarısız üniversiteyi failed_universities.txt dosyasına ekler"""
#     try:
#         with open("failed_universities.txt", "a", encoding="utf-8") as f:
#             timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#             if error_message:
#                 f.write(f"{timestamp} - <üniversite>{üni}</üniversite> - Hata: {error_message}\n")
#             else:
#                 f.write(f"{timestamp} - <üniversite>{üni}</üniversite>\n")
#         logging.info(f"{üni} başarısız üniversiteler listesine eklendi.")
#     except Exception as e:
#         logging.error(f"Başarısız üniversiteler dosyasına yazma hatası: {e}")

# def process_university(üni, başlangıç_yılı=2025):
#     """
#     Belirli bir üniversite için tez verilerini işler ve kaydeder.
    
#     Args:
#         üni (str): İşlenecek üniversite adı
#         başlangıç_yılı (int): Başlangıç yılı (varsayılan: 2025)
    
#     Returns:
#         bool: İşlem başarılı ise True, değilse False
#     """
#     logging.info(f"{'='*50}")
#     logging.info(f"{üni} üniversitesi işleniyor...")
#     logging.info(f"{'='*50}")
    
#     url = "https://tez.yok.gov.tr/UlusalTezMerkezi/"
#     driver = None
    
#     try:
#         # Data dizini kontrolü
#         if not os.path.exists("data"):
#             os.makedirs("data")
#             logging.info("'data' dizini oluşturuldu.")
        
#         # Driver oluştur ve arama yap
#         driver = get_driver(url)
#         sonuç = click_process(driver, üni)
        
#         if not sonuç:
#             logging.warning(f"{üni} için sonuç bulunamadı.")
#             add_to_failed_universities(üni, "Sonuç bulunamadı")
#             if driver:
#                 driver.close()
#             return False
            
#         logging.info(f"{üni} için toplam sonuç sayısı: {sonuç}")
        
#         # Sonuç sayısına göre işlem stratejisini belirle
#         if sonuç < 2000:
#             # 2000'den az sonuç - tek seferde işle
#             process_results(driver, üni)
#         else:
#             # 2000'den fazla sonuç - yıl aralıklarına böl
#             if driver:
#                 driver.close()
            
#             # Döngü parametrelerini belirle
#             if sonuç < 4000:
#                 yıl_aralığı_adımı = 12  # Her 12 yılda bir böl
#                 döngü_sayısı = sonuç // 2000 + 2
#             else:
#                 yıl_aralığı_adımı = 6   # Her 6 yılda bir böl
#                 döngü_sayısı = sonuç // 1000 + 2
                
#             process_by_year_ranges(üni, başlangıç_yılı, yıl_aralığı_adımı, döngü_sayısı, url)
            
#         logging.info(f"{üni} için işlem başarıyla tamamlandı.")
#         return True
        
#     except Exception as e:
#         error_msg = f"HATA: {üni} işlenirken bir hata oluştu: {e}"
#         logging.error(error_msg)
#         add_to_failed_universities(üni, str(e))
#         if driver:
#             try:
#                 driver.close()
#             except:
#                 pass
#         return False

# def process_results(driver, üni, yıl1=None, yıl2=None):
#     """
#     Arama sonuçlarını işler ve CSV'ye kaydeder.
    
#     Args:
#         driver: Selenium WebDriver nesnesi
#         üni (str): Üniversite adı
#         yıl1 (int, optional): Başlangıç yılı
#         yıl2 (int, optional): Bitiş yılı
    
#     Returns:
#         bool: İşlem başarılı ise True, değilse False
#     """
#     try:
#         # Tabloyu al
#         df = get_table(driver)
        
#         if df.empty:
#             logging.warning("Tablo verisi alınamadı.")
#             add_to_failed_universities(üni, "Tablo verisi alınamadı")
#             driver.close()
#             return False
            
#         logging.info(f"Toplam {len(df)} adet tez verisi alındı.")
        
#         # Dosya adını belirle
#         if yıl1 and yıl2:
#             filename = f"data/{üni}_{yıl1}_{yıl2}.csv"
#         else:
#             filename = f"data/{üni}.csv"
            
#         # CSV'ye kaydet
#         df.to_csv(filename, index=False)
#         logging.info(f"Veriler {filename} dosyasına kaydedildi.")
        
#         # Driver'ı kapat
#         driver.close()
#         return True
        
#     except Exception as e:
#         error_msg = f"Sonuçları işlerken hata: {e}"
#         logging.error(error_msg)
#         add_to_failed_universities(üni, error_msg)
#         try:
#             driver.close()
#         except:
#             pass
#         return False

# def process_by_year_ranges(üni, başlangıç_yılı, yıl_adımı, döngü_sayısı, url):
#     """
#     Belirli yıl aralıklarında bölünmüş şekilde verileri işler.
    
#     Args:
#         üni (str): Üniversite adı
#         başlangıç_yılı (int): Başlangıç yılı
#         yıl_adımı (int): Her adımda azaltılacak yıl sayısı
#         döngü_sayısı (int): Kaç yıl aralığı oluşturulacağı
#         url (str): YÖK tez merkezi URL'si
    
#     Returns:
#         bool: İşlem başarılı ise True, değilse False
#     """
#     current_year = başlangıç_yılı
#     success_count = 0
    
#     for i in range(1, döngü_sayısı):
#         # Yıl aralığını belirle
#         if i == döngü_sayısı - 1:
#             yıl1 = 1900  # Son aralık için 1900'den başla
#         else:
#             yıl1 = current_year - yıl_adımı
            
#         yıl2 = current_year
        
#         logging.info(f"\nYıl aralığı: {yıl1}-{yıl2} işleniyor...")
        
#         driver = None
#         try:
#             # Bu yıl aralığı için driver oluştur ve arama yap
#             driver = get_driver(url)
#             sonuç = click_process(driver, üni, yıl1=yıl1, yıl2=yıl2)
            
#             if not sonuç:
#                 logging.warning(f"{üni} için {yıl1}-{yıl2} aralığında sonuç bulunamadı.")
#                 if driver:
#                     driver.close()
#                 continue
                
#             logging.info(f"{üni} için {yıl1}-{yıl2} aralığında {sonuç} sonuç bulundu.")
            
#             # Sonuçları işle
#             if process_results(driver, üni, yıl1, yıl2):
#                 success_count += 1
        
#         except Exception as e:
#             error_msg = f"{üni} için {yıl1}-{yıl2} aralığında hata oluştu: {e}"
#             logging.error(error_msg)
#             add_to_failed_universities(üni, f"Yıl aralığı {yıl1}-{yıl2}: {str(e)}")
#             if driver:
#                 try:
#                     driver.close()
#                 except:
#                     pass
            
#         # Bir sonraki aralık için yılları güncelle
#         current_year = yıl1 - 1
    
#     logging.info(f"{üni} için {döngü_sayısı-1} aralıktan {success_count} tanesi başarıyla işlendi.")
#     return success_count > 0


# def main():
#     """Ana program akışı"""
#     # Log sistemi oluştur
#     log_file = setup_logging()
    
#     # failed_universities.txt dosyasının başına başlık ekle
#     try:
#         with open("failed_universities.txt", "w", encoding="utf-8") as f:
#             f.write(f"# Başarısız Üniversiteler - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
#             f.write("# Format: Tarih Saat - <üniversite>Üniversite Adı</üniversite> - Hata Mesajı\n")
#             f.write("-" * 80 + "\n")
#     except Exception as e:
#         logging.error(f"Failed universities dosyası oluşturma hatası: {e}")
    
#     try:
#         # Üniversite listesini oku
#         # üniversiteler = pd.read_csv("üniversiteler.csv")['üniversiteler'].tolist()
        
#         logging.info(f"Toplam {len(üniversiteler)} üniversite işlenecek.")
        
#         # Başarılı ve başarısız listeleri
#         başarılı_üniler = []
#         başarısız_üniler = []
        
#         # Her üniversite için işlem yap
#         for üni in üniversiteler:
#             if process_university(üni, başlangıç_yılı=2025):
#                 başarılı_üniler.append(üni)
#             else:
#                 başarısız_üniler.append(üni)
                
#             # Üniversiteler arası kısa bekleme
#             time.sleep(2)
        
#         # Özet log
#         logging.info(f"\n{'='*50}")
#         logging.info(f"İŞLEM ÖZET RAPORU")
#         logging.info(f"{'='*50}")
#         logging.info(f"Toplam üniversite sayısı: {len(üniversiteler)}")
#         logging.info(f"Başarılı işlenen üniversite sayısı: {len(başarılı_üniler)}")
#         logging.info(f"Başarısız üniversite sayısı: {len(başarısız_üniler)}")
        
#         if başarısız_üniler:
#             logging.warning("Başarısız üniversiteler:")
#             for üni in başarısız_üniler:
#                 logging.warning(f"<üniversite>{üni}</üniversite>")
            
#         # Özet dosyası
#         özet_file = f"logs/ozet_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
#         with open(özet_file, 'w', encoding='utf-8') as f:
#             f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
#             f.write(f"Log dosyası: {log_file}\n\n")
#             f.write(f"Toplam üniversite sayısı: {len(üniversiteler)}\n")
#             f.write(f"Başarılı işlenen üniversite sayısı: {len(başarılı_üniler)}\n")
#             f.write(f"Başarısız üniversite sayısı: {len(başarısız_üniler)}\n\n")
            
#             f.write("Başarılı üniversiteler:\n")
#             for i, üni in enumerate(başarılı_üniler, 1):
#                 f.write(f"{i}. {üni}\n")
            
#             f.write("\nBaşarısız üniversiteler:\n")
#             for i, üni in enumerate(başarısız_üniler, 1):
#                 f.write(f"{i}. <üniversite>{üni}</üniversite>\n")
        
#         logging.info(f"Özet rapor {özet_file} dosyasına kaydedildi.")
#         logging.info(f"Başarısız üniversiteler 'failed_universities.txt' dosyasına kaydedildi.")
#         logging.info("\nTüm üniversiteler için işlem tamamlandı!")
        
#     except Exception as e:
#         logging.error(f"Ana program hatası: {e}")
#         return False


# if __name__ == "__main__":
#     main()